# This notebook will compare different ML models with MAD

In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

d:\anaconda\envs\unimol_tools\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [17]:
# out-of-sample tasks 
test_1 = pd.read_csv('BHA_model_data/test1.csv',header=None)
test_2 = pd.read_csv('BHA_model_data/test2.csv',header=None)
test_3 = pd.read_csv('BHA_model_data/test3.csv',header=None)
test_4 = pd.read_csv('BHA_model_data/test4.csv',header=None)

Train_234 = pd.concat([test_2,test_3,test_4], ignore_index=False) # additive 1
Train_134 = pd.concat([test_1,test_3,test_4], ignore_index=False)
Train_124 = pd.concat([test_1,test_2,test_4], ignore_index=False)
Train_123 = pd.concat([test_1,test_2,test_3], ignore_index=False)


In [20]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.svm import LinearSVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.ensemble import AdaBoostRegressor
import utils_plot_yield as utils

In [32]:
models_need_fit = [
        #   AdaBoostRegressor(random_state=42, n_estimators=100),  
        #   LinearRegression(),
        #   LinearSVR(max_iter=1000),
        #   KNeighborsRegressor(n_neighbors=7),          
          RandomForestRegressor(n_estimators=50,random_state=42)#,
          #MLPRegressor(hidden_layer_sizes=(1000,500), activation='logistic', solver='sgd', max_iter=10000, random_state=42, alpha=0.0001, learning_rate_init=0.00002, batch_size = len(X_train))
         ]

r2 = []
rmse = []

Train = [Train_234, Train_134, Train_124, Train_123]
Test = [test_1, test_2, test_3, test_4]

for i in range(4):

    X_train = np.array(Train[i].iloc[:,0:-1])
    y_train = np.array(Train[i].iloc[:,-1]).flatten()
    X_test = np.array(Test[i].iloc[:,0:-1])
    y_test = np.array(Test[i].iloc[:,-1]).flatten()

    preds, r2_values, rmse_values  = utils.fit_models(X_train,
                                                    X_test,
                                                    y_train,
                                                    y_test,
                                                    models_need_fit)
    r2.append(r2_values)
    rmse.append(rmse_values)
    df_r2 = pd.DataFrame(r2,columns= models_need_fit)
    df_rmse = pd.DataFrame(rmse,columns=models_need_fit)
df = pd.concat([df_r2,df_rmse], axis=1, names=['R2','RMSE'])
df.to_csv('additive_randomforest.csv') 

RandomForestRegressor(n_estimators=50, random_state=42)
Done fitting models
RandomForestRegressor(n_estimators=50, random_state=42)
Done fitting models
RandomForestRegressor(n_estimators=50, random_state=42)
Done fitting models
RandomForestRegressor(n_estimators=50, random_state=42)
Done fitting models


In [30]:
Train = [Train_234, Train_134, Train_124, Train_123]
Test = [test_1, test_2, test_3, test_4]


X_train = np.array(Train[3].iloc[:,0:-1])
y_train = np.array(Train[3].iloc[:,-1]).flatten()
X_test = np.array(Test[3].iloc[:,0:-1])
y_test = np.array(Test[3].iloc[:,-1]).flatten()

preds, r2_values, rmse_values  = utils.fit_models(X_train,
                                                X_test,
                                                y_train,
                                                y_test,
                                                models_need_fit)
r2_values, rmse_values

AdaBoostRegressor(n_estimators=100, random_state=42)
LinearRegression()
LinearSVR()


d:\anaconda\envs\unimol_tools\lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
d:\anaconda\envs\unimol_tools\lib\site-packages\sklearn\svm\_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


KNeighborsRegressor(n_neighbors=7)
RandomForestRegressor(random_state=42)
Done fitting models


([0.5175457405708022,
  -4.8144051438519637e+24,
  -30.90692901705766,
  0.5831791351790255,
  0.5227962518819942],
 [18.37089634789537,
  58032808865906.98,
  149.39799786543387,
  17.075640366789838,
  18.27065837988394])

In [36]:
import xgboost as xgb

In [44]:
Train = [Train_234, Train_134, Train_124, Train_123]
Test = [test_1, test_2, test_3, test_4]

r2 = []
rmse = []

reg = xgb.XGBRegressor(
        tree_method="hist",
        n_estimators=128,
        n_jobs=16,
        max_depth=8,
        multi_strategy='one_output_per_tree',
        subsample=0.6,
    )

for i in range(4):

    X_train = np.array(Train[i].iloc[:,0:-1])
    y_train = np.array(Train[i].iloc[:,-1]).flatten()
    X_test = np.array(Test[i].iloc[:,0:-1])
    y_test = np.array(Test[i].iloc[:,-1]).flatten()

    reg.fit(X_train,y_train)
    preds = reg.predict(X_test)
    r_squared = r2_score(y_test, preds)
    rmse = mean_squared_error(y_test, preds) ** 0.5
     
    print(r_squared)
    print(rmse)
    


0.7838008460417127
12.687032572793361
0.7522100489828267
13.485462574189343
0.6797841893904342
15.909563845852539
0.5415519348419822
17.908010748914645
